In [3]:
from numpy import *

U, feature_mat, VT = linalg.svd([[1,1],[7,7]])
print VT


[[-0.70710678 -0.70710678]
 [ 0.70710678 -0.70710678]]


In [18]:
test1 = [[1,1,1,0,0],
        [2,2,2,0,0],
        [1,1,1,0,0],
        [5,5,5,0,0],
        [1,1,0,2,2],
        [0,0,0,3,3],
        [0,0,0,1,1]]
U1,feature_mat1, VT1 = linalg.svd(test1)
#print VT1[:3,:]
feature_mat1_del = mat([[feature_mat1[0],0,0],[0,feature_mat1[1],0],[0,0, feature_mat1[2]]])
print U1[:,:3] *feature_mat1_del* VT1[:3,:]

[[  1.00000000e+00   1.00000000e+00   1.00000000e+00   7.75989921e-16
    7.71587483e-16]
 [  2.00000000e+00   2.00000000e+00   2.00000000e+00   3.00514919e-16
    2.77832253e-16]
 [  1.00000000e+00   1.00000000e+00   1.00000000e+00   2.18975112e-16
    2.07633779e-16]
 [  5.00000000e+00   5.00000000e+00   5.00000000e+00   3.00675663e-17
   -1.28697294e-17]
 [  1.00000000e+00   1.00000000e+00  -5.48397422e-16   2.00000000e+00
    2.00000000e+00]
 [  3.21319929e-16   4.43562065e-16  -3.48967188e-16   3.00000000e+00
    3.00000000e+00]
 [  9.71445147e-17   1.45716772e-16  -1.52655666e-16   1.00000000e+00
    1.00000000e+00]]


In [28]:
from numpy import linalg as la

def eucli_similary(vec1, vec2):
    return 1.0 / 1.0 + la.norm(vec1 - vec2)

def pear_similary(vec1, vec2):
    return 0.5 + 0.5 * corrcoef(vec1, vec2, rowvar=0)[0][1]

def cos_similary(vec1, vec2):
    A_B = float(vec1.T * vec2)
    denom = la.norm(vec1) * la.norm(vec2)
    return 0.5 + 0.5 * (A_B / denom)
test1_mat = mat(test1)
print 'pear_similary: ', pear_similary(test1_mat[:,1], test1_mat[:,2])
print 'eucli_similary: ',eucli_similary(test1_mat[:,1], test1_mat[:,2])
print 'pear_similary: ', cos_similary(test1_mat[:,1], test1_mat[:,2])


 pear_similary:  0.988982626009
eucli_similary:  2.0
pear_similary:  0.992125492126


In [64]:
def compute_est(data_mat, user, sim_method, item):
    row_num = shape(data_mat)[0]
    total_sim = 0.0
    rating_similary = 0.0
    for m in range(row_num):
        user_rating = data_mat[user,m]
        if user_rating == 0 or m == item:
            continue
        together_eval_users = nonzero(logical_and(data_mat[:, m].A > 0, data_mat[:, item].A > 0))[0]
        if len(together_eval_users) == 0:
            similary = 0
        else:
            similary = sim_method(data_mat[together_eval_users,m], data_mat[together_eval_users,item])
        total_sim += similary
        rating_similary += similary * user_rating
        if total_sim == 0:
            return 0
        else:
            return rating_similary / float(total_sim)
        

def recommend(data_mat, user, sim_method=cos_similary, est_method=compute_est, N=3):
    not_evaluate_items = nonzero(data_mat[user,:].A == 0)[1] # row
    if len(not_evaluate_items) == 0:
        return 'you rate everything!'
    items_scores = [] # alse use {}
    for j in range(len(not_evaluate_items)):
        estim_score = est_method(data_mat, user, sim_method, not_evaluate_items[j])
        items_scores.append((not_evaluate_items[j], estim_score))
    return sorted(items_scores, key=lambda item: item[1], reverse=True)[:N]


   

In [65]:
test2 = [[4,4,0,2,2],
        [4,0,0,3,3],
        [4,0,0,1,1],
        [1,1,1,2,0],
        [1,1,1,0,0],
        [5,5,5,0,0]]
test2_mat = mat(test2)
print recommend(test2_mat, 3)


[(4, 1.0)]


In [80]:
def svd_estim(data_mat, user,sim_method, item):
    row_num = shape(data_mat)[0]
    total_sim = 0.0
    rating_similary = 0.0
    U, featute_mat, VT = la.svd(data_mat)
    feature_mat_4 = mat(eye(4) * feature_mat[:4])
    svd_mat = data_mat.T * U[:,:4] * feature_mat_4.I
    for m in range(row_num):
        user_rating = data_mat[user,m]
        if user_rating == 0 or m == item:
            continue

        similary = sim_method(svd_mat[m,:].T, svd_mat[item,:].T)
        total_sim += similary
        rating_similary += similary * user_rating
        if total_sim == 0:
            return 0
        else:
            return rating_similary / float(total_sim)

In [82]:
print recommend(test2_mat,2w, est_method=svd_estim)

[(1, 4.0), (2, 4.0)]


In [75]:
U, feature_mat, VT = la.svd(test2_mat)
print mat(eye(4) * feature_mat[:4])

[[ 11.47878945   0.           0.           0.        ]
 [  0.           5.82127116   0.           0.        ]
 [  0.           0.           2.58384698   0.        ]
 [  0.           0.           0.           1.92648388]]


In [88]:
def print_mat(data_mat, thresh=0.8):
    for i in range(32):
        for j in range(32):
            if float(data_mat[i,j]) > 0.8:
                print 1 ,
            else: 
                print 0,
        print ' '

def img_compress(num_of_feat_v=3, thresh=0.8):
    origin_data = []
    f = open('0_5.txt')
    for line in f.readlines():
        temp_str = line.strip()
        temp_l = [int(i) for i in temp_str]
        origin_data.append(temp_l)
    origin_data_mat = mat(origin_data)
    print '---------------------------------------------------------'
    print_mat(origin_data_mat)
    
    U, feature_v, VT = la.svd(origin_data_mat)
    sig_com = mat(zeros((num_of_feat_v, num_of_feat_v)))
    for m in range(num_of_feat_v):
        sig_com[m,m] = feature_v[m]
    svd_mat = U[:,:num_of_feat_v] * sig_com * VT[:num_of_feat_v,:]
    print '***********************************************************'
    print_mat(svd_mat)
    
img_compress()

---------------------------------------------------------
0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0  
0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0  
0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0  
0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0  
0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0  
0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0  
0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0  
0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0  
0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0  
0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0  
0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0  
0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0  
0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0  
0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0  
0 0 0 0 0 0 1 1 1 